In [0]:
pip install mlflow

  Obtaining dependency information for mlflow from https://files.pythonhosted.org/packages/72/f1/be329ef23da1933135afc5e493cbe1b7ecf5302890f2a7ac2124b64c01bd/mlflow-2.21.3-py3-none-any.whl.metadata
  Using cached mlflow-2.21.3-py3-none-any.whl.metadata (30 kB)
  Obtaining dependency information for mlflow-skinny==2.21.3 from https://files.pythonhosted.org/packages/2d/f8/b71f88ca373f248fd7fdf3751f74c7b36a71b7ee2b5f4b803ee053ac963a/mlflow_skinny-2.21.3-py3-none-any.whl.metadata
  Using cached mlflow_skinny-2.21.3-py3-none-any.whl.metadata (31 kB)
  Obtaining dependency information for Flask<4 from https://files.pythonhosted.org/packages/af/47/93213ee66ef8fae3b93b3e29206f6b251e65c97bd91d8e1c5596ef15af0a/flask-3.1.0-py3-none-any.whl.metadata
  Using cached flask-3.1.0-py3-none-any.whl.metadata (2.7 kB)
  Obtaining dependency information for Jinja2<4,>=2.11 from https://files.pythonhosted.org/packages/62/a1/3d680cbfd5f4b8f15abc1d571870c5fc3e594bb582bc3b64ea099db13e56/jinja2-3.1.6-py3-none-a

In [0]:
import mlflow

mlflow.set_experiment("/Users/yq2397@columbia.edu/take-home-exercise-3-yq2397")

2025/04/09 06:02:17 INFO mlflow.tracking.fluent: Experiment with name '/Users/yq2397@columbia.edu/take-home-exercise-3-yq2397' does not exist. Creating a new experiment.


<Experiment: artifact_location='dbfs:/databricks/mlflow-tracking/108253902178481', creation_time=1744178537510, experiment_id='108253902178481', last_update_time=1744178537510, lifecycle_stage='active', name='/Users/yq2397@columbia.edu/take-home-exercise-3-yq2397', tags={'mlflow.experiment.sourceName': '/Users/yq2397@columbia.edu/take-home-exercise-3-yq2397',
 'mlflow.experimentType': 'MLFLOW_EXPERIMENT',
 'mlflow.ownerEmail': 'yq2397@columbia.edu',
 'mlflow.ownerId': '3797474588272739'}>

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import DoubleType
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import RandomForestRegressor, GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [0]:
df_results = spark.read.csv('s3://columbia-gr5069-main/raw/results.csv', header = True)
df_drivers = spark.read.csv('s3://columbia-gr5069-main/raw/drivers.csv', header = True)
df_races= spark.read.csv('s3://columbia-gr5069-main/raw/races.csv', header = True)

In [0]:
merged_df = df_results.join(df_races, on="raceId", how="left")
merged_df = merged_df.join(df_drivers, on="driverId", how="left")
display(merged_df.limit(10))

driverId,raceId,resultId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId,year,round,circuitId,name,date,time,url,fp1_date,fp1_time,fp2_date,fp2_time,fp3_date,fp3_time,quali_date,quali_time,sprint_date,sprint_time,driverRef,number,code,forename,surname,dob,nationality,url
1,18,1,1,22,1,1,1,1,10,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1,2008,1,1,Australian Grand Prix,2008-03-16,04:30:00,http://en.wikipedia.org/wiki/2008_Australian_Grand_Prix,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton
2,18,2,2,3,5,2,2,2,8,58,+5.478,5696094,41,3,1:27.739,217.586,1,2008,1,1,Australian Grand Prix,2008-03-16,04:30:00,http://en.wikipedia.org/wiki/2008_Australian_Grand_Prix,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,heidfeld,\N,HEI,Nick,Heidfeld,1977-05-10,German,http://en.wikipedia.org/wiki/Nick_Heidfeld
3,18,3,3,7,7,3,3,3,6,58,+8.163,5698779,41,5,1:28.090,216.719,1,2008,1,1,Australian Grand Prix,2008-03-16,04:30:00,http://en.wikipedia.org/wiki/2008_Australian_Grand_Prix,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,rosberg,6,ROS,Nico,Rosberg,1985-06-27,German,http://en.wikipedia.org/wiki/Nico_Rosberg
4,18,4,4,5,11,4,4,4,5,58,+17.181,5707797,58,7,1:28.603,215.464,1,2008,1,1,Australian Grand Prix,2008-03-16,04:30:00,http://en.wikipedia.org/wiki/2008_Australian_Grand_Prix,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,alonso,14,ALO,Fernando,Alonso,1981-07-29,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso
5,18,5,1,23,3,5,5,5,4,58,+18.014,5708630,43,1,1:27.418,218.385,1,2008,1,1,Australian Grand Prix,2008-03-16,04:30:00,http://en.wikipedia.org/wiki/2008_Australian_Grand_Prix,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,kovalainen,\N,KOV,Heikki,Kovalainen,1981-10-19,Finnish,http://en.wikipedia.org/wiki/Heikki_Kovalainen
6,18,6,3,8,13,6,6,6,3,57,\N,\N,50,14,1:29.639,212.974,11,2008,1,1,Australian Grand Prix,2008-03-16,04:30:00,http://en.wikipedia.org/wiki/2008_Australian_Grand_Prix,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,nakajima,\N,NAK,Kazuki,Nakajima,1985-01-11,Japanese,http://en.wikipedia.org/wiki/Kazuki_Nakajima
7,18,7,5,14,17,7,7,7,2,55,\N,\N,22,12,1:29.534,213.224,5,2008,1,1,Australian Grand Prix,2008-03-16,04:30:00,http://en.wikipedia.org/wiki/2008_Australian_Grand_Prix,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,bourdais,\N,BOU,Sébastien,Bourdais,1979-02-28,French,http://en.wikipedia.org/wiki/S%C3%A9bastien_Bourdais
8,18,8,6,1,15,8,8,8,1,53,\N,\N,20,4,1:27.903,217.180,5,2008,1,1,Australian Grand Prix,2008-03-16,04:30:00,http://en.wikipedia.org/wiki/2008_Australian_Grand_Prix,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,raikkonen,7,RAI,Kimi,Räikkönen,1979-10-17,Finnish,http://en.wikipedia.org/wiki/Kimi_R%C3%A4ikk%C3%B6nen
9,18,9,2,4,2,\N,R,9,0,47,\N,\N,15,9,1:28.753,215.100,4,2008,1,1,Australian Grand Prix,2008-03-16,04:30:00,http://en.wikipedia.org/wiki/2008_Australian_Grand_Prix,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,kubica,88,KUB,Robert,Kubica,1984-12-07,Polish,http://en.wikipedia.org/wiki/Robert_Kubica
10,18,10,7,12,18,\N,R,10,0,43,\N,\N,23,13,1:29.558,213.166,3,2008,1,1,Australian Grand Prix,2008-03-16,04:30:00,http://en.wikipedia.org/wiki/2008_Australian_Grand_Prix,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N,glock,\N,GLO,Timo,Glock,1982-03-18,German,http://en.wikipedia.org/wiki/Timo_Glock


In [0]:
merged_df = merged_df.withColumn("grid", merged_df["grid"].cast(DoubleType()))
merged_df = merged_df.withColumn("positionOrder", merged_df["positionOrder"].cast(DoubleType()))
merged_df = merged_df.withColumn("laps", merged_df["laps"].cast(DoubleType()))

merged_df = merged_df.withColumn("dob", F.to_date(merged_df["dob"]))
merged_df = merged_df.withColumn("date", F.to_date(merged_df["date"]))
merged_df = merged_df.withColumn("driver_age", 
                                F.datediff(merged_df["date"], merged_df["dob"])/365.25)

model_df = merged_df.select("grid", "driver_age", "laps", "positionOrder")
model_df = model_df.dropna()

print(f"Total records for modeling: {model_df.count()}")
display(model_df.limit(10))

Total records for modeling: 26080


grid,driver_age,laps,positionOrder
1.0,23.186858316221766,58.0,1.0
5.0,30.850102669404517,58.0,2.0
7.0,22.718685831622178,58.0,3.0
11.0,26.63107460643395,58.0,4.0
3.0,26.406570841889117,58.0,5.0
13.0,23.175906913073238,57.0,6.0
17.0,29.045859000684462,55.0,7.0
15.0,28.413415468856947,53.0,8.0
2.0,23.27173169062286,47.0,9.0
18.0,25.9958932238193,43.0,10.0


In [0]:
train_df, test_df = model_df.randomSplit([0.8, 0.2], seed=42)
print(f"Training set count: {train_df.count()}")
print(f"Testing set count: {test_df.count()}")

feature_cols = ["grid", "driver_age", "laps"]
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

Training set count: 20874
Testing set count: 5206


In [0]:
def train_model_simple(model_type, params=None):
    
    with mlflow.start_run():
        mlflow.log_param("model_type", model_type)
        if params:
            for key, value in params.items():
                mlflow.log_param(key, value)

        train_vector = assembler.transform(train_df)
        test_vector = assembler.transform(test_df)

        if model_type == 'rf':
            model = RandomForestRegressor(featuresCol="features", labelCol="positionOrder", **params if params else {})
        elif model_type == 'gbt':
            model = GBTRegressor(featuresCol="features", labelCol="positionOrder", **params if params else {})

        trained_model = model.fit(train_vector)

        predictions = trained_model.transform(test_vector)

        evaluator = RegressionEvaluator(labelCol="positionOrder", predictionCol="prediction")
        rmse = evaluator.setMetricName("rmse").evaluate(predictions)
        r2 = evaluator.setMetricName("r2").evaluate(predictions)

        pred_df = predictions.select("positionOrder", "prediction").toPandas()
        acc_within_1 = np.mean(np.abs(pred_df["positionOrder"] - np.round(pred_df["prediction"])) <= 1) * 100
        acc_within_3 = np.mean(np.abs(pred_df["positionOrder"] - np.round(pred_df["prediction"])) <= 3) * 100

        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("accuracy_within_1", acc_within_1)
        mlflow.log_metric("accuracy_within_3", acc_within_3)

        feature_importance = pd.DataFrame({
            'Feature': feature_cols,
            'Importance': trained_model.featureImportances.toArray()
        }).sort_values('Importance', ascending=False)
        
        plt.figure(figsize=(8, 4))
        plt.barh(feature_importance['Feature'], feature_importance['Importance'])
        plt.title('Feature Importance')
        plt.tight_layout()
        plt.savefig("feature_importance.png")
        plt.close()

        mlflow.log_artifact("feature_importance.png")
        feature_importance.to_csv("feature_importance.csv", index=False)
        mlflow.log_artifact("feature_importance.csv")
        
        mlflow.spark.log_model(trained_model, f"{model_type}_model")
        
        return trained_model, rmse, r2, acc_within_1, acc_within_3

In [0]:
# Parameters
rf_params = [
    {'numTrees': 100, 'maxDepth': 10, 'seed': 42},
    {'numTrees': 200, 'maxDepth': 15, 'seed': 42},
    {'numTrees': 300, 'maxDepth': 5, 'seed': 42},
    {'numTrees': 100, 'maxDepth': 20, 'seed': 42},
    {'numTrees': 200, 'maxDepth': 10, 'seed': 42}
]

gbt_params = [
    {'maxIter': 100, 'stepSize': 0.1, 'maxDepth': 3, 'seed': 42},
    {'maxIter': 200, 'stepSize': 0.05, 'maxDepth': 5, 'seed': 42},
    {'maxIter': 100, 'stepSize': 0.01, 'maxDepth': 7, 'seed': 42},
    {'maxIter': 200, 'stepSize': 0.1, 'maxDepth': 5, 'seed': 42},
    {'maxIter': 300, 'stepSize': 0.05, 'maxDepth': 3, 'seed': 42}
]

results = []

# Random Forest experiments
for i, params in enumerate(rf_params):
    print(f"RF Experiment {i+1}/{len(rf_params)}")
    _, rmse, r2, acc1, acc3 = train_model_simple('rf', params)
    results.append({
        'Model Type': 'Random Forest',
        'Parameters': params,
        'RMSE': rmse,
        'R²': r2,
        'Accuracy within ±1': acc1,
        'Accuracy within ±3': acc3
    })

# Gradient Boosting experiments
for i, params in enumerate(gbt_params):
    print(f"GBT Experiment {i+1}/{len(gbt_params)}")
    _, rmse, r2, acc1, acc3 = train_model_simple('gbt', params)
    results.append({
        'Model Type': 'Gradient Boosting',
        'Parameters': params,
        'RMSE': rmse,
        'R²': r2,
        'Accuracy within ±1': acc1,
        'Accuracy within ±3': acc3
    })


results_df = pd.DataFrame(results)
best_model = results_df.loc[results_df['RMSE'].idxmin()]

print("\n===== Experiment Results =====")
print(results_df[['Model Type', 'RMSE', 'R²', 'Accuracy within ±1', 'Accuracy within ±3']])
print("\n===== Best Model =====")
print(f"Model Type: {best_model['Model Type']}")
print(f"Parameters: {best_model['Parameters']}")
print(f"RMSE: {best_model['RMSE']:.4f}")
print(f"R²: {best_model['R²']:.4f}")
print(f"Accuracy within ±1 position: {best_model['Accuracy within ±1']:.2f}%")
print(f"Accuracy within ±3 positions: {best_model['Accuracy within ±3']:.2f}%")

RF Experiment 1/5


2025/04/09 06:13:37 INFO mlflow.spark: Inferring pip requirements by reloading the logged model from the databricks artifact repository, which can be time-consuming. To speed up, explicitly specify the conda_env or pip_requirements when calling log_model().
2025/04/09 06:14:01 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: dbfs:/databricks/mlflow-tracking/108253902178481/de1add0519a44d57879755665be551cb/artifacts/rf_model/sparkml, flavor: spark). Fall back to return ['pyspark==3.5.0']. Set logging level to DEBUG to see the full traceback. 
2025/04/09 06:14:01 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.21.3, required: mlflow==2.11.4)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


RF Experiment 2/5


2025/04/09 06:15:51 INFO mlflow.spark: Inferring pip requirements by reloading the logged model from the databricks artifact repository, which can be time-consuming. To speed up, explicitly specify the conda_env or pip_requirements when calling log_model().
2025/04/09 06:16:14 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: dbfs:/databricks/mlflow-tracking/108253902178481/d984e884b2e940daa7e889b612c9bc6d/artifacts/rf_model/sparkml, flavor: spark). Fall back to return ['pyspark==3.5.0']. Set logging level to DEBUG to see the full traceback. 
2025/04/09 06:16:14 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.21.3, required: mlflow==2.11.4)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


RF Experiment 3/5


...

GBT Experiment 4/5


2025/04/09 06:28:24 INFO mlflow.spark: Inferring pip requirements by reloading the logged model from the databricks artifact repository, which can be time-consuming. To speed up, explicitly specify the conda_env or pip_requirements when calling log_model().
2025/04/09 06:28:48 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: dbfs:/databricks/mlflow-tracking/108253902178481/9a5223846a5a488283f777d3637655af/artifacts/gbt_model/sparkml, flavor: spark). Fall back to return ['pyspark==3.5.0']. Set logging level to DEBUG to see the full traceback. 
2025/04/09 06:28:48 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.21.3, required: mlflow==2.11.4)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.

GBT Experiment 5/5


2025/04/09 06:30:54 INFO mlflow.spark: Inferring pip requirements by reloading the logged model from the databricks artifact repository, which can be time-consuming. To speed up, explicitly specify the conda_env or pip_requirements when calling log_model().
2025/04/09 06:31:18 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: dbfs:/databricks/mlflow-tracking/108253902178481/ef6827f3d3f34e79936a26cf1f6dce45/artifacts/gbt_model/sparkml, flavor: spark). Fall back to return ['pyspark==3.5.0']. Set logging level to DEBUG to see the full traceback. 
2025/04/09 06:31:18 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.21.3, required: mlflow==2.11.4)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


===== Experiment Results =====
          Model Type      RMSE  ...  Accuracy within ±1  Accuracy within ±3
0      Random Forest  4.286192  ...           28.082981           62.120630
1      Random Forest  4.292913  ...           28.447945           62.255090
2      Random Forest  4.528787  ...           23.107952           53.169420
3      Random Forest  4.322660  ...           28.697657           61.928544
4      Random Forest  4.284113  ...           27.814061           61.736458
5  Gradient Boosting  4.248023  ...           29.811756           63.772570
6  Gradient Boosting  4.285469  ...           30.464848           63.484441
7  Gradient Boosting  4.309558  ...           29.850173           63.734153
8  Gradient Boosting  4.345866  ...           30.426431           62.773723
9  Gradient Boosting  4.238328  ...           30.061468           63.676527

[10 rows x 5 columns]

===== Best Model =====
Model Type: Gradient Boosting
Parameters: {'maxIter': 300, 'stepSize': 0.05, 'maxDept

**Best Model**

For this F1 position prediction project, we testes 10 different models: 5 Random Forest and 5 Gradient Boosting models with different hyperparameters. The best performing model was a Gradient Boosting Regressor with 300 trees, a learning rate of 0.05, and a maximum depth of 3. This model had an RMSE of 4.24 and could explain about 69% of what determines a driver's finishing position. In simpler terms, our model could predict a driver's finishing position within 1 spot of their actual position 30% of the time, and within 3 spots 64% of the time. This beat all our Random Forest models. We think the model worked well because we didn't make the trees too deep or the learning too aggressive, which helped it avoid memorizing the training data instead of learning real patterns.